# Day 08 – Unity Catalog Governance

##  Create Catalog

In [0]:
%sql
CREATE CATALOG ecommerce_catalog;

##  Creating Schemas (Bronze,Silver & Gold) under catelog

In [0]:
%sql
CREATE SCHEMA ecommerce_catalog.bronze;
CREATE SCHEMA ecommerce_catalog.silver;
CREATE SCHEMA ecommerce_catalog.gold;

## Bronze Layer – Raw Data Governance
- Bronze is where raw data lands exactly as received from source systems (e.g., Kafka, S3, ADLS, vendor files).

## What we have in Bronze layer :
- No business logic
- No data correction
- Only schema enforcement + traceability

## Ingest Raw November Events (Bronze)

## Why governance matters here

- Tracks who accessed raw customer data
- Prevents accidental deletion
- Enables lineage

## Creating Volume In Unity Catelog bronze layer

In [0]:
%sql
CREATE VOLUME ecommerce_catalog.bronze.bronze_nov;

In [0]:
df_bronze = spark.read.csv('/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv', header = True, inferSchema = True)

df_bronze.write.format("delta")\
    .mode("overwrite")\
    .save('/Volumes/ecommerce_catalog/bronze/bronze_nov')

## Creating Delta Tables in bronze layer

In [0]:
df_bronze.write.format("delta").saveAsTable("ecommerce_catalog.bronze.events_nov")

##  Silver Layer – Clean & Trusted Data
- Silver layer is where data engineers prepare data for consumption.

## Clean & Standardize Events :
- Null user IDs
- Invalid event types

In [0]:
%sql
select distinct event_type from ecommerce_catalog.bronze.events_nov

In [0]:
%sql
CREATE TABLE ecommerce_catalog.silver.events_nov_clean AS
SELECT
    event_time,
    event_type,
    product_id,
    category_id,
    category_code,
    brand,
    price,
    user_id,
    user_session
FROM ecommerce_catalog.bronze.events_nov
WHERE user_id IS NOT NULL
  AND event_type IN ('view', 'cart', 'purchase');


## Permissions on Silver Data

In [0]:
%sql
GRANT SELECT
ON TABLE ecommerce_catalog.silver.events_nov_clean
TO `abdulsql5@gmail.com`;

GRANT ALL PRIVILEGES
ON TABLE ecommerce_catalog.silver.events_nov_clean
TO `abdulsql5@gmail.com`;

## Gold Layer – Business & Analytics Access

### Gold is what business users actually consume:
- Dashboards
- Reports
- ML features


In [0]:
%sql
CREATE VIEW ecommerce_catalog.gold.vw_events_nov_analytics AS
SELECT
    event_time,
    event_type,
    product_id,
    category_id,
    category_code,
    brand,
    price
FROM ecommerce_catalog.silver.events_nov_clean;

In [0]:
%sql
GRANT SELECT
ON VIEW ecommerce_catalog.gold.vw_events_nov_analytics
TO `abdulsql5@gmail.com`;

## Purchase-Only View (Business KPI)
- Finance & growth teams care about revenue-driving events only.

In [0]:
%sql
CREATE VIEW ecommerce_catalog.gold.vw_purchases_nov AS
SELECT *
FROM ecommerce_catalog.silver.events_nov_clean
WHERE event_type = 'purchase';